# Fiona y Shapely

[Fiona](https://github.com/Toblerity/Fiona) es una biblioteca de Python para la lectura y escritura de datos geoespaciales basada en [OGR](https://www.gdal.org/ogr_utilities.html).

Por su parte, [Shapely](https://pypi.org/project/Shapely/) es un paquete de Python enfocado en la manipulación y el análisis de objetos planos. Está basado en la bibliotecas [GEOS](https://trac.osgeo.org/geos/) y [JTS](https://projects.eclipse.org/projects/locationtech.jts).

## Instalación

Para instalar ambos paquetes mediante **conda**, debe ejecutarse la siguiente instrucción desde la línea de comandos de Anaconda:

```
conda install gdal fiona shapely
```

## Importación

In [ ]:
import fiona
# Impresión de la versión de Fiona
print("Versión de Fiona:", fiona.__version__)

import shapely
# Impresión de la versión de Shapely
print("Versión de Shapely:", shapely.__version__)

## Ejemplos de uso

Recorrido de los registros de una capa vectorial e impresión de atributos

In [ ]:
with fiona.open('datos/provincias_snit_crtm05.geojson', 'r') as input:
    for input_record in input:
        print(input_record['properties']['nom_prov'])  

**Ejercicio**: Junto al nombre de la provincia, imprima el código de esta.

Impresión de geometrías

In [ ]:
from shapely.geometry import mapping, shape

with fiona.open('datos/us-states.json', 'r') as input:
    for input_record in input:
        print(input_record['geometry'])

Funciones geoespaciales

In [ ]:
from shapely.geometry import shape

with fiona.open('datos/provincias_snit.geojson', 'r') as input:
    for input_record in input:
        print(shape(input_record['geometry']).centroid)

In [ ]:
from shapely.geometry import mapping, shape

with fiona.open('datos/provincias_snit.geojson', 'r') as input:
    for input_record in input:
        print(mapping(shape(input_record['geometry']).centroid)['coordinates'][0], 
              mapping(shape(input_record['geometry']).centroid)['coordinates'][1])

**Ejercicio**: Junto a las coordenadas de cada centroide, imprima el nombre de la provincia.

**Ejercicio:** Imprima también el área de cada provincia, calculada mediante Shapely.

## Escritura de archivos

En el siguiente ejemplo, se recorren los registros de un archivo vectorial de polígonos, se obtienen sus centroides y se escriben en un archivo separado.

In [ ]:
import fiona
from shapely.geometry import shape
from collections import OrderedDict
import logging

with fiona.open('datos/provincias_snit.geojson', 'r') as input:
    
    output_schema = {
        'geometry': 'Point',
        'properties': OrderedDict([
            ('provincia', 'str')
        ])
    }    
    
    with fiona.open('datos/centroides_provincias.geojson', 'w',
                    crs=input.crs, 
                    driver="GeoJSON",
                    schema=output_schema
                    ) as output:
        
        for input_record in input:
            try:
                output_record = {
                    'geometry': {
                        'type': 'Point',
                        'coordinates': ((mapping(shape(input_record['geometry']).centroid))['coordinates'][0],
                                        (mapping(shape(input_record['geometry']).centroid))['coordinates'][1])
                    },
                    'properties': OrderedDict([
                        ('provincia', input_record['properties']['nom_prov'])
                    ])
                }
                output.write(output_record)        

                print(shape(input_record['geometry']).centroid)
            except:
                logging.exception("Error al procesar el registro %s:", input_record['id'])
            

En el siguiente ejemplo, se determina el polígono en el que está ubicado cada punto de una capa

In [ ]:
import fiona
from shapely.geometry import shape
from collections import OrderedDict
import logging

with fiona.open('datos/especies.geojson', 'r') as points:
    
    with fiona.open('datos/curridabat-distritos-wgs84-ign-2019.geojson', 'r') as polys:
        
        for input_point in points:
            for input_poly in polys:
                if shape(input_poly['geometry']).contains(shape(input_point['geometry'])):
                    print(shape(input_point['geometry']), input_poly['properties']['nom_distr'])
                    break